In [1]:
import sys, os, distutils.core
sys.path.insert(0, os.path.abspath('../../../detectron2'))
sys.path.insert(0, os.path.abspath('../../../'))
sys.path.insert(0, os.path.abspath('.'))
os.environ['DETECTRON2_DATASETS'] = '/home/liwa/data/datasets/'

from detectron2.data import DatasetCatalog, MetadataCatalog
from oneformer.data.datasets.register_yeast_panoptic_annos_semseg import register_all_yeast_panoptic_annos_sem_seg
from tqdm import trange

/home/liwa/miniconda3/envs/oneformer/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from detectron2.config import get_cfg
from detectron2.projects.panoptic_deeplab import (
    PanopticDeeplabDatasetMapper,
    add_panoptic_deeplab_config,
)
from detectron2.engine import default_setup
from detectron2.engine import DefaultPredictor
from prediction import Predictor
from google.colab.patches import cv2_imshow
import os, json, cv2, random
from detectron2.utils.visualizer import Visualizer
import torch

In [3]:
torch.get_num_threads()
torch.set_num_threads(8)


In [4]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:8700m"
cfg = get_cfg()
add_panoptic_deeplab_config(cfg)
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file("/home/liwa/data/oneformer_output/output/config.yaml")

cfg.MODEL.DEVICE = "cpu"
# into cpu model
cfg.MODEL.SEM_SEG_HEAD.NORM = "BN"
cfg.MODEL.INS_EMBED_HEAD.NORM = "BN"
cfg.MODEL.RESNETS.NORM = "BN"


# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.PANOPTIC_DEEPLAB.TOP_K_INSTANCE = None
cfg.MODEL.PANOPTIC_DEEPLAB.INSTANCE_AREA_THRESHOLD = 2000


# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = os.path.abspath("/home/liwa/data/oneformer_output/output/model_0059999.pth")
# cfg.MODEL.WEIGHTS = "detectron2://DeepLab/R-52.pkl"
# cfg.INPUT.MIN_SIZE_TEST = 512
# cfg.INPUT.MAX_SIZE_TRAIN = 2048

In [7]:
predictor.cfg

CfgNode({'VERSION': 2, 'MODEL': CfgNode({'LOAD_PROPOSALS': False, 'MASK_ON': False, 'KEYPOINT_ON': False, 'DEVICE': 'cpu', 'META_ARCHITECTURE': 'PanopticDeepLab', 'WEIGHTS': '/home/liwa/data/oneformer_output/output/model_0059999.pth', 'PIXEL_MEAN': [102.364, 102.364, 102.364], 'PIXEL_STD': [23.532, 23.532, 23.532], 'BACKBONE': CfgNode({'NAME': 'build_resnet_deeplab_backbone', 'FREEZE_AT': 0}), 'FPN': CfgNode({'IN_FEATURES': [], 'OUT_CHANNELS': 256, 'NORM': '', 'FUSE_TYPE': 'sum'}), 'PROPOSAL_GENERATOR': CfgNode({'NAME': 'RPN', 'MIN_SIZE': 0}), 'ANCHOR_GENERATOR': CfgNode({'NAME': 'DefaultAnchorGenerator', 'SIZES': [[32, 64, 128, 256, 512]], 'ASPECT_RATIOS': [[0.5, 1.0, 2.0]], 'ANGLES': [[-90, 0, 90]], 'OFFSET': 0.0}), 'RPN': CfgNode({'HEAD_NAME': 'StandardRPNHead', 'IN_FEATURES': ['res4'], 'BOUNDARY_THRESH': -1, 'IOU_THRESHOLDS': [0.3, 0.7], 'IOU_LABELS': [0, -1, 1], 'BATCH_SIZE_PER_IMAGE': 256, 'POSITIVE_FRACTION': 0.5, 'BBOX_REG_LOSS_TYPE': 'smooth_l1', 'BBOX_REG_LOSS_WEIGHT': 1.0, '

In [5]:
from skimage.io import imread, imsave
import numpy as np
from pathlib import Path

In [6]:
predictor = Predictor(cfg)

In [7]:
movie_path_list = list(Path("/home/liwa/data/mating_data/patch/20240427_hm_scd2_hp_scd2/").rglob("*_rigid.tif"))
movie_path_list = [str(x) for x in movie_path_list]

In [8]:
movie_path_list

['/home/liwa/data/mating_data/patch/20240427_hm_scd2_hp_scd2/20240427_yml21_44_amp_04_3_R3D_merge_rigid.tif',
 '/home/liwa/data/mating_data/patch/20240427_hm_scd2_hp_scd2/20240427_yml21_44_amp_04_6_R3D_merge_rigid.tif',
 '/home/liwa/data/mating_data/patch/20240427_hm_scd2_hp_scd2/20240427_yml21_44_amp_04_5_R3D_merge_rigid.tif',
 '/home/liwa/data/mating_data/patch/20240427_hm_scd2_hp_scd2/20240427_yml21_44_amp_04_2_R3D_merge_rigid.tif']

In [10]:
movie = imread(movie_path_list[0])

In [ ]:
for movie_path in movie_path_list:
    print(movie_path)
    movie = imread(movie_path)
    segment_movie = np.zeros(movie.shape[:3], dtype=np.uint16)
    for frame in trange(0, movie.shape[0]):
        image = movie[frame,:,:,0]
        image = image - image.min()
        image = image / image.max()
        image = (image * 255).astype(np.uint8)
        im = np.stack((image,)*3, axis=-1)
        prediction_output = predictor(im)
        panoptic_seg, _ = prediction_output["panoptic_seg"]
        segment_movie[frame] = np.array(panoptic_seg)
    imsave(movie_path[:-4]+"_mask.tif", segment_movie)

/home/liwa/data/mating_data/patch/20240427_hm_scd2_hp_scd2/20240427_yml21_44_amp_04_3_R3D_merge_rigid.tif


  0%|                                                   | 0/881 [00:00<?, ?it/s]/home/liwa/miniconda3/envs/oneformer/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180588308/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|█████████████████████████████████████| 881/881 [42:01:32<00:00, 171.73s/it]
/tmp/ipykernel_3336414/816345210.py:14: UserWarning: /home/liwa/data/mating_data/patch/20240427_hm_scd2_hp_scd2/20240427_yml21_44_amp_04_3_R3D_merge_rigid_mask.tif is a low contrast image
  imsave(movie_path[:-4]+"_mask.tif", segment_movie)


/home/liwa/data/mating_data/patch/20240427_hm_scd2_hp_scd2/20240427_yml21_44_amp_04_6_R3D_merge_rigid.tif


100%|███████████████████████████████████████████████████████████████████████████████████████████| 881/881 [41:53:37<00:00, 171.19s/it]
/tmp/ipykernel_3336414/816345210.py:14: UserWarning: /home/liwa/data/mating_data/patch/20240427_hm_scd2_hp_scd2/20240427_yml21_44_amp_04_6_R3D_merge_rigid_mask.tif is a low contrast image
  imsave(movie_path[:-4]+"_mask.tif", segment_movie)


/home/liwa/data/mating_data/patch/20240427_hm_scd2_hp_scd2/20240427_yml21_44_amp_04_5_R3D_merge_rigid.tif


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 881/881 [42:41:56<00:00, 174.48s/it]
/tmp/ipykernel_3336414/816345210.py:14: UserWarning: /home/liwa/data/mating_data/patch/20240427_hm_scd2_hp_scd2/20240427_yml21_44_amp_04_5_R3D_merge_rigid_mask.tif is a low contrast image
  imsave(movie_path[:-4]+"_mask.tif", segment_movie)


/home/liwa/data/mating_data/patch/20240427_hm_scd2_hp_scd2/20240427_yml21_44_amp_04_2_R3D_merge_rigid.tif


 48%|███████████████████████████████████████████████████████████████████████████▋                                                                                  | 422/881 [20:26:56<22:05:21, 173.25s/it]

In [27]:
imsave(movie_path[:-4]+"_mask.tif", segment_movie)

/tmp/ipykernel_2980070/708227417.py:1: UserWarning: /home/liwa/data/mating_data/patch/20240607_2scd2_gfp_002/1_rigid_mask.tif is a low contrast image
  imsave(movie_path[:-4]+"_mask.tif", segment_movie)


In [13]:
im = imread("./00087-1.png")
# segment_movie = np.zeros(movie.shape[:3], dtype=np.uint16)
# for frame in trange(0, movie.shape[0]):
#     image = movie[frame,:,:,0]
#     image = image - image.min()
#     image = image / image.max()
#     image = (image * 255).astype(np.uint8)
#     im = np.stack((image,)*3, axis=-1)

In [ ]:
prediction_output = predictor(im)
print(prediction_output.keys())

/home/liwa/miniconda3/envs/oneformer/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180588308/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [11]:
panoptic_seg, segments_info = prediction_output["panoptic_seg"]
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
cv2_imshow(out.get_image()[:, :, ::-1])

NameError: name 'prediction_output' is not defined

In [1]:
panoptic_seg.shape

NameError: name 'panoptic_seg' is not defined